In [1]:
import googlemaps
import geojsoncontour
import numpy as np
import pandas as pd
import random
import shapefile
import shapely

from datetime import datetime, timedelta
from scipy import interpolate
import scipy as sp
import scipy.ndimage
from shapely import geometry

from matplotlib import cm
import json
import os

import folium
import folium.plugins

In [2]:
conf = json.load(open("config.json"))

In [4]:
FORHOME = datetime(2020, 2, 12, *conf["forhome"])
FORWORK = datetime(2020, 2, 12, *conf["forwork"])
WORKADDY = conf["work_address"]
gmaps = googlemaps.Client(key=conf["gmap_key"])

dc_shapefile = shapefile.Reader("Washington_DC_Boundary.shp")
dc_shape = dc_shapefile.shapes()[0]
dc_geometry = geometry.shape(dc_shape)

In [5]:
def d_matrix(origins, destinations, leavetime):
    m = gmaps.distance_matrix(origins=origins, destinations=destinations,
                                      mode='driving', units='imperial', departure_time=leavetime,
                                      traffic_model='best_guess')
    time.sleep(5)
    return m

In [6]:
def gen_points(n):
    minx, miny, maxx, maxy = dc_shape.bbox
    locations = []
    while len(locations) < n:
        pnt = geometry.Point(random.uniform(minx, maxx), random.uniform(miny, maxy))
        if dc_geometry.contains(pnt):
            locations.append(pnt)
    return locations

In [7]:
def divide_chunks(l, n=100): 
    for i in range(0, len(l), n):  
        yield l[i:i + n]
        
def get_times(drive_times):
    times = []
    for drive_time in drive_times: 
        for row in drive_time['rows']:
            data = row['elements']
            for elem in data:
                try:
                    times.append(elem['duration_in_traffic']['value']/60)
                except:
                    pass
    return times


In [8]:
if not os.path.exists("times.csv"):    
    locations = gen_points(n=1000)  # use 1000 pts to interpolate a grid
    origins_coords_y = [location.y for location in locations]
    origins_coords_x = [location.x for location in locations]
    origins = list(zip(origins_coords_y, origins_coords_x))
    destinations = [WORKADDY]
    drive_times = [d_matrix(split_origin, destinations, FORWORK) for split_origin in divide_chunks(origins)]
    times = get_times(drive_times)
    df = pd.DataFrame({"y": origins_coords_y, "x": origins_coords_x,"time": times})
    df.to_csv("times.csv")
else:
    df = pd.read_csv("times.csv")

In [10]:
# contourf = plt.contourf(x_mesh, y_mesh, z_mesh, levels=5, alpha=0.5, colors=['#d7191c',  '#fdae61',  '#ffffbf',  '#abdda4',  '#2b83ba'],
#                         linestyles='None', vmin=min(times), vmax=max(times))

In [11]:
# geojson = geojsoncontour.contourf_to_geojson(
#     contourf=contourf,
#     min_angle_deg=3.0,
#     ndigits=5,
#     stroke_width=1,
#     fill_opacity=0.5)

In [13]:
n = 100
minx, miny, maxx, maxy = dc_shape.bbox
def is_dc(x, y):
    pnt = geometry.Point(x, y)
    return dc_geometry.contains(pnt)

bmask = [[is_dc(x, y) 
             for x in np.linspace(minx, maxx, n)]             
             for y in np.linspace(miny, maxy, n)] 
mask = np.array(bmask, dtype=float)

In [20]:
def gen_ipoints(df, n):
    """generate interpolated points from a set of given points"""
    minx, miny, maxx, maxy = dc_shape.bbox
    x, y = np.linspace(minx, maxx, n), np.linspace(miny, maxy, n)
    xx, yy = np.meshgrid(x, y)
    
    points = df[["x","y"]].values
    return sp.interpolate.griddata(points, df["time"], (xx, yy), method="cubic")


img = gen_ipoints(df, 100)

img = np.minimum(img, df["time"].max())

img = np.maximum(img, df["time"].min())


img = img - np.nanmin(img)

#img = np.log(img)
img = img / np.nanmax(img)
# mask values we want off are zero, dividing by zero results in inf and we can adjust a colormap with that
img = img / mask  

#img = np.nan_to_num(img, 0)
img = img[::-1, :]


def cmap(x):
    mp = cm.get_cmap("jet")
    if np.isnan(x) or np.isinf(x):
        return (0, 0, 0, 0)
    else:
        return mp(x)

m = folium.Map(location=[38.9072,-77.0369], tiles='Stamen Toner', zoom_start=12)
minx, miny, maxx, maxy = dc_shape.bbox
folium.raster_layers.ImageOverlay(
    img, 
    bounds=[[miny, minx], [maxy, maxx]], 
    opacity=.75,
    colormap=cmap).add_to(m)
#m.save("commute.html")
m

/Users/pbutler/.virtualenvs/dchousing/lib/python3.7/site-packages/ipykernel_launcher.py:23: RuntimeWarning: divide by zero encountered in true_divide
/Users/pbutler/.virtualenvs/dchousing/lib/python3.7/site-packages/ipykernel_launcher.py:23: RuntimeWarning: invalid value encountered in true_divide
